# Introduction to Semantic Kernel

## Theory Module

### Prerequisites

- Python 3.8 or later
- Basic understanding of Python programming
- OpenAI API key or Azure OpenAI access
- Semantic Kernel library installed

In [1]:
# Setup - call this once at the beginning of the notebook to add the parent directory to the path

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

### 1. What is Semantic Kernel?

Semantic Kernel is an open-source SDK that integrates LLMs into applications. Think of it as a bridge between your application and AI models, providing a structured way to:

- Create AI-powered functions
- Combine AI capabilities with traditional programming
- Manage context and memory
- Orchestrate complex AI workflows

Compared to langchain or llama-index Semantic Kernel focuses deeply on workflows.

### Setting up the Kernel

In Semantic Kernel all interactions with a LLM happens through a kernel.
Let's connect to it!

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai import OpenAIProtocol

# Create a kernel - the central orchestrator in Semantic Kernel
kernel = sk.Kernel()

# Configure the kernel with an AI service
kernel.add_text_completion_service(
    "gpt-4",  # Service ID
    "OpenAI"  # Service type
)

# Test the kernel
print("Kernel initialized successfully!")

Key concepts to understand about the kernel:

- It's your main entry point for all SK operations
- Manages AI service connections
- Orchestrates function execution
- Handles memory and context

### 2. Skills


Skills in Semantic Kernel are collections of related functions.

There are two types of skills/functions:

**Native Skills**: Traditional Python functions





In [ ]:
class MathSkill:
    """A simple native skill for mathematical operations"""
    
    def add(self, a: float, b: float) -> float:
        """Adds two numbers together"""
        return a + b
    
    def multiply(self, a: float, b: float) -> float:
        """Multiplies two numbers"""
        return a * b

# Register the skill with the kernel
math_skill = kernel.import_skill(MathSkill(), "math")

# Use the skill
result = math_skill.add(5, 3)
print(f"5 + 3 = {result}")

**Semantic Skills**: AI-powered functions defined by prompts

In [ ]:
# Define a semantic function
prompt_template = """
Input: {{$input}}
Task: Generate three creative names for a pet {{$animal_type}}.
Requirements:
- Names should be family-friendly
- Each name should be unique
- Include a brief explanation for each name

Output your response in this format:
1. [Name] - [Explanation]
2. [Name] - [Explanation]
3. [Name] - [Explanation]
"""

# Create the semantic function
pet_names = kernel.create_semantic_function(
    prompt_template,
    function_name="generate_pet_names",
    description="Generates creative pet names"
)

# Example usage
async def generate_names():
    result = await pet_names.invoke(input="", animal_type="cat")
    print(result)

Of course you can combine both types

In [ ]:
class TextProcessingSkill:
    """A skill for processing text"""
    
    def count_words(self, input: str) -> int:
        """Counts words in a text"""
        return len(input.split())
    
    def format_results(self, count: int, summary: str) -> str:
        """Formats the analysis results"""
        return f"Word count: {count}\nSummary: {summary}"

# Create a semantic function for summarization
summarize_prompt = """
Input: {{$input}}
Task: Create a concise one-sentence summary.
"""

summarize = kernel.create_semantic_function(
    summarize_prompt,
    function_name="summarize"
)

# Register the native skill
text_skill = kernel.import_skill(TextProcessingSkill(), "text")

# Create a pipeline
async def analyze_text(input_text: str):
    # First, get the summary
    summary = await summarize.invoke(input_text)
    
    # Count words in original text
    word_count = text_skill.count_words(input_text)
    
    # Format results
    return text_skill.format_results(word_count, str(summary))

### 3. Memory

Semantic Kernel provides built-in memory capabilities using embeddings:


In [ ]:
# Setup memory
from semantic_kernel.memory.memory_store import MemoryStore
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory

memory_store = MemoryStore()
memory = SemanticTextMemory(storage=memory_store)
kernel.import_skill(memory, "memory")

# Store information
async def store_memory():
    await memory.save_information(
        collection="facts",
        id="fact1",
        text="Semantic Kernel was created by Microsoft.",
        description="Basic SK fact"
    )

# Retrieve information
async def recall_memory():
    results = await memory.search("Who created Semantic Kernel?")
    print(results[0].text if results else "No information found")

### 4. Wrapping it up - Emoji Translator

In [ ]:
class EmojiNarratorSkill:
    """A skill that translates complex narratives into and from emoji sequences"""
    
    def __init__(self, kernel: sk.Kernel):
        # Create encoder function
        encode_template = """
        Story: {{$story}}
        Style: {{$style}}
        
        Transform this story into a sequence of emojis that:
        1. Captures key plot points
        2. Represents character emotions
        3. Includes relevant symbols for setting
        4. Uses emoji combinations for complex concepts
        5. Maintains narrative flow
        
        Also provide a legend explaining your emoji choices.
        """
        
        # Create decoder function
        decode_template = """
        Emoji sequence: {{$emoji_sequence}}
        Genre: {{$genre}}
        Tone: {{$tone}}
        
        Create a detailed story from these emojis that:
        1. Interprets each emoji creatively
        2. Builds a coherent narrative
        3. Adds unexpected but logical connections
        4. Includes dialogue and descriptions
        5. Matches the specified genre and tone
        """
        
        self.encoder = kernel.create_semantic_function(encode_template)
        self.decoder = kernel.create_semantic_function(decode_template)
    
    async def story_to_emoji(self, story: str, style: str = "modern"):
        return await self.encoder.invoke(story=story, style=style)
    
    async def emoji_to_story(self, emoji_sequence: str, genre: str, tone: str):
        return await self.decoder.invoke(
            emoji_sequence=emoji_sequence,
            genre=genre,
            tone=tone
        )

# Example usage
async def emoji_story_demo():
    narrator = EmojiNarratorSkill(kernel)
    
    # Encode a story
    original_story = "A scientist accidentally creates a time machine but it only works on houseplants"
    emoji_version = await narrator.story_to_emoji(original_story, "sci-fi comedy")
    
    # Decode back to a different story
    new_story = await narrator.emoji_to_story(
        str(emoji_version),
        genre="philosophical thriller",
        tone="mysterious"
    )
    
    print(f"Original: {original_story}\nEmoji: {emoji_version}\nNew Story: {new_story}")